# CI Portfolio Project 5 - Filter Maintenance Predictor 2022
## **Feature Engineering Notebook**

## Objectives

**1. Cleaning**

Performed within the [data cleaning notebook](https://github.com/roeszler/filter-maintenance-predictor/blob/main/jupyter_notebooks/02_DataCleaning.ipynb).

**2. Data Transformation**
* Processing the data for the modelling stage.
* Transform data into a format that is useful for the algorithm learn the relationship among the variables.
* Evaluate the use of the following approaches to engineer the variables:
    * ordinal categorical encoding
    * numerical transformation
    * smart correlated selection
    
**3. Feature Extraction**
* Evenly distribute dust type

**4. Feature Selection**

**5. Feature Iteration**



### Inputs

1. Cleaned Test Dataset : `outputs/datasets/collection/dfCleanTrain.csv`

2. Cleaned Train Dataset : `outputs/datasets/collection/dfCleanTrain.csv`

### Outputs

* Generate engineered Train and Test sets, both saved under `outputs/datasets/transformed`

### Conclusions

  * Best approach to engineer variables based on...
  * Transformations that we will consider in our pipeline are...

---

# Change working directory

In [ ]:
import os
current_dir = os.getcwd()
current_dir

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("Current directory set to new location")

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Load Cleaned Data

In [ ]:
import pandas as pd
df_train = pd.read_csv(f'outputs/datasets/cleaned/dfCleanTrain.csv')
df_test = pd.read_csv(f'outputs/datasets/cleaned/dfCleanTest.csv')

---

# Data Transformation

## Ordinal Categorical encoding

#### Convert `Data_No` to a categorical variable

In [ ]:
data_no_total = df_total['Data_No'].map(str)
df_total['Data_No'] = data_no_total
df_total.info()

## Numerical transformation

This process can consider transformers like:
* Smooth with SMA
* Logarithmic in base e
* Logarithmic in base 10
* Reciprocal
* Power
* BoxCox
* Yeo Johnson


#### Outliers in differential pressure observations

In each bin we notice that the `differential_pressure` and `change_DP` observations, the size and direction of first few observations indicated they may be outliers. Considering three main methods to deal with outliers:
* Smooth with Simple Mean Average (SMA)
* Log transformation.
* Dropping the outliers.
* Winsorize method.

### Smoothing of **Differential Pressure** with a Simple Moving Average

The change in **differential pressure** that is opposite to the general trend of the data is indicated by **change_DP**. At these observations we see the recorded differential pressure value considerably more than the previous value and opposite to the trend. We have set the tolerance to identify this as a ±200% change in value, however this can be altered depending on what we see when fitting the models.

To smooth the variability of this measure we can apply a simple average value (SMA) of the last two measures, that moves as the observations progress. This will soften the change and reduce the instances of values that are higher / lower than the general trend. It will effectively reduce variability in the **differential pressure** measure.

<!-- Where:
* _df.loc[df['column name'] condition, 'new column name'] = 'value if condition is met'_
* _df['new column name'] = df['column name'].apply(lambda x: 'value if condition is met' if x condition else 'value if condition is not met')_ -->

The following calculates **Simple Moving Average** and engineers it to be included as a **smoothed version** of `differential_pressure`. Here, any change greater than **±200%** from the previous value is replaced with the moving mean of the surrounding values.

In [ ]:
sma = (df_test['Differential_pressure'].shift(1) + df_test['Differential_pressure'].shift(-1))/2
df_test.insert(loc=8, column='sma_DP', value=sma)

for n in df_test:
    if (df_test['change_DP'] < -2).any().any():
        if (df_test['change_DP'] > 2).any().any():
            df_test['sma_DP'] = (df_test['Differential_pressure'].shift(1) + df_test['Differential_pressure'].shift(0))/2

check_dn = [df_test.Data_No != df_test.Data_No.shift(1)].copy()
df_test['sma_DP'].check_dn = df_test['Differential_pressure']

df_test.insert(loc=2, column='smoothed_DP', value=sma)
df_test['smoothed_DP'] = df_test['Differential_pressure']
df_test.loc[df_test['change_DP'] < -2, 'smoothed_DP'] = df_test['sma_DP']
df_test.loc[df_test['change_DP'] > 2, 'smoothed_DP'] = df_test['sma_DP']
del df_test['sma_DP']

df_test.insert(loc=9, column='change_SMA', value=sma)
df_test['change_SMA'] = df_test['smoothed_DP'].diff().fillna(0)
df_test['change_SMA'].check_dn = df_test['smoothed_DP']
df_test.loc[df_test['change_SMA'].check_dn == False, 'change_SMA'] = df_test['smoothed_DP']

# df_test.loc[363:368]
df_test.loc[36405:36410]

In [ ]:
df_test.smoothed_DP.loc[36405:36410]

In [ ]:
df_test['change_SMA'].loc[36405:36410]

In [ ]:
df_test[df_test.Data_No != df_test.Data_No.shift(1)].head()

In [ ]:
df_test[df_test.Data_No != df_test.Data_No.shift(-1)].tail()

### Logarithmic Transformation of Differential Pressure

As seen at data collection, the original continuous `differential_pressure` data is right or **positively skewed** at 1.81 and does not follow the shape of a normally distributed bell curve.

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.histplot(x = df_train['Differential_pressure'])
plt.show()

A log transformation of this data to make it as “normal” as possible so that the statistical analysis results from this data become more valid.

In [ ]:
df_train_log = df_train['Differential_pressure']
log_dp = np.log(df_train_log)
# df_train['log_DP'] = log_dp
sns.histplot(x = log_dp)
# sns.boxplot(x = log_dp)
plt.title('Log_DP Histogram')
plt.show()

**Observations** 
* The shape of the numerical differential_pressure data is improved by a natural logarithmic transformation
    * Untreated values of 78.23 mean, std.deviation at 107.34 and median at 35.17, indicates the positively skewed nature of the original data
    * Transformed the data is much improved with mean 3.18 much closer to the median 3.89

The transformed data is still affected by the slight negative skew in the data:
* Pressure measures show unusual behavior at the start of most tests, producing an unusual entry tail of values at zero or below.
    * These can be managed by adding a constant to stop the value becoming negative, or indicate the negative as a missing number or drop the entire observation.
    * Considering these measures indicate a zone at the beginning of the test procedure to 'equalize' the difference in pressure between the areas before and after the filter, we can treat them as outliers and confidently drop the entire row this observation sits in.

### Dropping Outliers
Dropping all negative numbers to reduce the noise of the data

#### Evaluation of detecting outliers using IQR

In [ ]:
df_train_IQR = df_train.copy()

# IQR
Q1 = np.percentile(df_train_IQR, 25, method='midpoint')
Q3 = np.percentile(df_train_IQR, 75,method='midpoint')
IQR = Q3 - Q1
print("Old Shape: ", df_train_IQR.shape)
 
# Upper bound
upper = np.where(df_train_IQR >= (Q3+1.5*IQR))
# Lower bound
lower = np.where(df_train_IQR <= (Q1-1.5*IQR))
 
''' Removing the Outliers '''
df_train_IQR.drop(upper[0], inplace = True)
df_train_IQR.drop(lower[0], inplace = True)
 
print("New Shape: ", df_train_IQR.shape)

df_train_log_IQR = df_train_IQR['Differential_pressure']
log_dp_IQR = np.log(df_train_log_IQR)
# df_train_IQR['log_DP'] = log_dp
sns.histplot(x = log_dp_IQR)
plt.title('IQR Histogram')
plt.show()

IQR transformation is not particularly effective on this data. We can observe that it has made the log data more negatively skewed, in the exact opposite direction we started with. We will not apply this method and proceed with other techniques.

#### The Winsorize Method
Winsorization is the process of replacing the extreme values of statistical data in order to limit the effect of the outliers on the calculations or the results obtained by using that data. To apply this to a our `differential_pressure` variable, where outliers are present only at one end of the data:
* The lower 10% values of the data will have their values set equal to the value of the data point at the 10th percentile.

In [ ]:
from scipy.stats.mstats import winsorize

WinsorizedArrayMean = np.mean(df_train['Differential_pressure'])
df_train['winz_Mean'] = winsorize(df_train['Differential_pressure'],(0.1,0.1))
WinsorizedArray = df_train['winz_Mean']
plt.boxplot(WinsorizedArray)
plt.title('Winsorized array')
plt.show()
WinsorizedArrayNewMean = np.mean(WinsorizedArray)
print('Old Mean: ', WinsorizedArrayMean)
print('New Mean: ', WinsorizedArrayNewMean)

In [ ]:
df_train

### Remove rows with negative numbers

In [ ]:
df_train.loc[:,'log_DP'] = log_dp
df_train = df_train[df_train.select_dtypes(include=[np.number]).ge(0).all(1)]
df_train

In [ ]:
old_shape = pd.read_csv(f'outputs/datasets/cleaned/dfCleanTrain.csv')
old_shape['winz_Mean'] = WinsorizedArray
old_shape['log_DP'] = np.log(df_train['Differential_pressure'])
sns.histplot(x = df_train['log_DP'])
plt.title('Log_DP Histogram')
plt.show()
print("Old Shape: ", old_shape.shape)
print("New Shape: ", df_train.shape)

## Smart correlated selection

---

# Feature Extraction

## Evenly distribute dataset by `Dust` type

Both the **train** and **test** sets supplied have data distributed unevenly between 50 test bins. To account for this we wish to assess the measures of central tendency for each Dust class, with the aim of reducing the data size to a more evenly proportioned one between classes.

#### **Train** Dataset

**Considerations**

* The proportion of data that **has reached filter failure** is represented by how close `filter_balance` is to zero or less. 
    * Data with filter_balance values approaching zero may be worth keeping and will make part of our heuristic decision process.
* Notwithstanding that the **mean** is the most frequently used measure of central tendency because it uses all values in the data set to give you an average
    * For data from skewed distributions (like `differential_pressure`), the **median** is better than the mean because it isn’t influenced by extremely large values.

In the following calculation, we see a summary of the top ten `Data_No` bins where `differential_pressure` observations that have made it to the **600 Pa** (the point of filter failure).

In [ ]:
last_row_train = df_train[df_train.Data_No != df_train.Data_No.shift(-1)]
# last_row_descending = last_row_train.sort_values(by='Dust', ascending=True)
last_row_descending = last_row_train.sort_values(by='Differential_pressure', ascending=False)
last_row_descending.head(n=10)

Note the diagram below showing proportions of `Dust` variable in the **df_train** dataset.
* It shows a disproportionate mix between classes. This will be the first dataset we tidy up.

In [ ]:
%matplotlib inline

category_totals = df_train.groupby('Dust')['Differential_pressure'].count().sort_values()
category_totals.plot(kind="barh", title='Proportion of Dust Classes in df_train\n', xlabel='\nObservations', ylabel='Dust Class')
category_totals

#### Our next aim is to 
* Fill these bins with data that best represents a central tendency.
* Make the size of each bin around ±**5000** observations (similar to the A4 Coarse Dust class bin) 

#### Procedure
* Include a comparison to how far each `differential_pressure` measure **deviates** or how far it is from the **.median()** value of the bin.
* Ordered by `filter_balance` showing sets with data closest to 600 Pa `differential_pressure`.
* Include comparison to median
* Add a cumulative measure of Data_No's to use as a ranking
* Create a dataframe of the A3 Medium Dust : **1.025**

Add a calculation of Standard Deviation to **df_train** test set

In [ ]:
del df_train['winz_Mean']
del df_train['log_DP']
std_group = df_train.groupby('Data_No').std()
std_group.index.name = None
std_group.loc[:,'Data_No'] = std_group.index
map_std = df_train['Data_No'].map(std_group.set_index('Data_No')['Differential_pressure'])
df_train.loc[:,'std_DP'] = map_std
# df_test.loc[363:368]
df_train.loc[444:453]

Add a calculation of Coefficient of Variation (variance) to **df_train** test set

In [ ]:
import numpy as np
cv = lambda data: np.std(data, ddof=1) / np.mean(data, axis=0) * 100 
var_group = df_train.groupby('Data_No').apply(cv)
var_group.index.name = None
var_group.loc[:,'Data_No'] = var_group.index
map_var = df_train['Data_No'].map(var_group.set_index('Data_No')['Differential_pressure'])
df_train.loc[:,'cv_DP'] = map_var
# df_test.loc[363:368]
df_train.loc[444:453]

The coefficient of variation is an indication of how far the standard deviation is away from the mean. As we can see it does not add value to our understanding of the data, primarily due the the skewed nature of the `differential_pressure` continuous variable. 
* This re-enforces the understanding that descriptive statistics using the mean may not be preferred measure of central tendency.

**Remove Coefficient of Variation and Add Median to df_train**
* Median is the preferred measure of central tendency to observe in a skewed dataset such as this as it is not as affected by larger values.

In [ ]:
del df_train['cv_DP']
median_group = df_train.groupby('Data_No').median()
median_group.index.name = None
median_group.loc[:,'Data_No'] = median_group.index
map_median = df_train['Data_No'].map(median_group.set_index('Data_No')['Differential_pressure'])
df_train.loc[:,'median_DP'] = map_median
df_train.loc[444:453]

#### Now we can evaluate the dataframe with just **A3 Dust** in it, ordered by `filter_balance` as a measure of time to filter failure

Map the size of each bin and include a cumulative sum of each **bin size** to help see which data bin that reaches **4500** or more total values.

In [ ]:
bin_sum = df_train.groupby('Data_No')['Data_No'].count().reset_index(name='bin_Tot')
map_bin = df_train['Data_No'].map(bin_sum.set_index('Data_No')['bin_Tot'])
df_train.loc[:, 'bin_Size'] = map_bin
# df_train.loc[38817:38827]
dust_A3 = df_train[df_train['Dust'] == 1.025]
filter_A3 = dust_A3[dust_A3.Data_No != dust_A3.Data_No.shift(-1)]
df_train_A3 = filter_A3.sort_values(by='Filter_Balance', ascending=True)
df_train_A3['c_Sum'] = df_train_A3['bin_Size'].cumsum()
df_train_A3.head(13)

We can see that in the current dataframe containing only A3 Medium Dust observations, that is ordered by those tests with closest to a completed test to failure:
* **The top 9 data bins (seen at bin 7) would extract a A3 Medium dust training dataset with 5,109 observations**
* We will now perform a further PDA to evaluate the suitability of these further

#### Rank by Standard Deviations, ordered by `std_DP`
The standard deviation is used to measure the spread of values in a sample.

In [ ]:
# dust_A3 = df_train[df_train['Dust'] == 1.025]
# filter_A3 = dust_A3[dust_A3.Data_No != dust_A3.Data_No.shift(-1)]
df_train_A3_std = filter_A3.sort_values(by='std_DP', ascending=True)
df_train_A3_std['c_Sum'] = df_train_A3_std['bin_Size'].cumsum()
df_train_A3_std.head(15)

#### Rank by central tenancy of the Median value, ordered by `median_DP`
* the value of the number in the middle of the dataset

In [ ]:
# dust_A3 = df_train[df_train['Dust'] == 1.025]
# filter_A3 = dust_A3[dust_A3.Data_No != dust_A3.Data_No.shift(-1)]
df_train_A3_median = filter_A3.sort_values(by='median_DP', ascending=True)
df_train_A3_median['c_Sum'] = df_train_A3_median['bin_Size'].cumsum()
df_train_A3_median.head(14)

#### Considerations
* T...

---

### Extract these bins from the df_train dataset

Make a separate frame indicating the bin numbers we wish to extract

In [ ]:
bin_no = df_train_A3['Data_No'].head(9)
bin_no.to_frame()

Use these references to create a dataframe `df_train_cleaned_A3` that is ready for inclusion in our final dataframe `df_train_clean`.
* Note we disregard the cumulative sum measure as it doesn't add value to further calculations

In [ ]:
df_train_copy = df_train
df_train_cleaned_A3 = df_train_copy[df_train_copy['Data_No'].isin(bin_no)]
df_train_cleaned_A3

#### A Quick Review: 

In [ ]:
print('Shape started with: ', dust_A3.shape)
print('Shape we have now: ', df_train_cleaned_A3.shape)

In [ ]:
df_train_cleaned_A3

Add these to a new dataset to compare

In [ ]:
dust_A2 = df_train[df_train['Dust'] == 0.900]
dust_A3 = df_train_cleaned_A3
dust_A4 = df_train[df_train['Dust'] == 1.200]

df_train_compare = pd.concat([dust_A2, dust_A3, dust_A4], ignore_index = True)
df_train_compare

In [ ]:
%matplotlib inline

category_totals = df_train_compare.groupby('Dust')['Differential_pressure'].count().sort_values()
category_totals.plot(kind="barh", title='Proportion of Dust Classes in df_train_compare\n', xlabel='\nObservations', ylabel='Dust Class')
category_totals

---

### Repeat procedure with remaining `Dust` classes and **Test** dataset

Extract, Clear and Replace...

In [ ]:
# bin_sum = df_train.groupby('Data_No')['Data_No'].count().reset_index(name='bin_Tot')
# map_bin = df_train['Data_No'].map(bin_sum.set_index('Data_No')['bin_Tot'])
# df_train.loc[:, 'bin_Size'] = map_bin

# df_train.loc[38817:38827]

dust_A2 = df_train[df_train['Dust'] == 0.900]
filter_A2 = dust_A2[dust_A2.Data_No != dust_A2.Data_No.shift(-1)]
df_train_A2 = filter_A2.sort_values(by='Filter_Balance', ascending=True)
df_train_A2['c_Sum'] = df_train_A2['bin_Size'].cumsum()
df_train_A2.head(13)

In [ ]:
bin_no = df_train_A2['Data_No'].head(9)
bin_no.to_frame()

In [ ]:
df_train_copy = df_train
df_train_cleaned_A2 = df_train_copy[df_train_copy['Data_No'].isin(bin_no)]
df_train_cleaned_A2

In [ ]:
dust_A2 = df_train_cleaned_A2
dust_A3 = df_train_cleaned_A3
dust_A4 = df_train[df_train['Dust'] == 1.200]

df_train_compare = pd.concat([dust_A2, dust_A3, dust_A4], ignore_index = True)
df_train_compare

In [ ]:
%matplotlib inline

category_totals = df_train_compare.groupby('Dust')['Differential_pressure'].count().sort_values()
category_totals.plot(kind="barh", title='Proportion of Dust Classes in df_train_compare\n', xlabel='\nObservations', ylabel='Dust Class')
category_totals

### How much data do we need?
* At a bin sample of 9 for both A2 and A3 dust, we currently have a little under 15,000 observations to train the ML Models
* Will review this depending on the performance of the models
* Overfitting with too much data vs underfitting with too little is a balance and the qualitative nature of the heuristics surrounding the predictions made
* Nonlinear Algorithms (like clustering) may need more data

Short answer is, **we have plenty of extra data at this point that may be less suited to training the model**. 

Notwithstanding, it is still live data and may there may be some value toward increasing the power of our ML Models. 
We do have the option to alter the selected number of dust class bins and possibly augmenting the smaller A4 dust dataset with the **Synthetic Minority Oversampling Technique (SMOTE)** or SMOTE NC for categorical data should we need.

In [ ]:
df_train_cleaned = df_train_compare
df_train_cleaned

---

# Feature Selection

---

# Feature Iteration

---

## Divide Datasets

Divide **df_total** into original **df_test** & **df_train** proportions as supplied

In [ ]:
data_no_total = df_total['Data_No'].map(int).round(decimals=0)
df_total['Data_No'] = data_no_total
n = df_total['Data_No'][0:len(df_total)]
df_train = df_total[n < 51].reset_index(drop=True, names='index')
df_test = df_total[n > 50].reset_index(drop=True, names='index')
del df_train['RUL']
df_test

In [ ]:
df_train

## Save Datasets 
Save the files to /transformed folder

In [ ]:
import os
try:
  os.makedirs(name='outputs/datasets/transformed')
except Exception as e:
  print(e)

df_train.to_csv(f'outputs/datasets/transformed/dfTransformedTrain.csv',index=False)
df_test.to_csv(f'outputs/datasets/transformed/dfTransformedTest.csv',index=False)
df_total.to_csv(f'outputs/datasets/transformed/dfTransformedTotal.csv',index=False)

---

# Conclusions and Next steps

#### Conclusions: 
* 

#### Next Steps:
* Regression Model
* Classification Model
* Cluster Model
* Correlation Study